In [8]:
import os, sys
project_root = os.path.abspath(os.path.join(os.path.dirname('../')))
sys.path.append(project_root)

from deepface.basemodels import VGGFace
tf_face = VGGFace.loadModel()

from deepface.extendedmodels import Gender
tf_gender = Gender.loadModel()

from core.face.gender import Gender as PTGender
pt_gender = PTGender()

from core.face.vggface import VGGFace as PTVGGFace
torch_model = PTVGGFace()
# import torch
# from core.face.vggface import VGGFace as PTVGGFace
# torch_model = PTVGGFace()

# weights_file = os.path.join(project_root, "pretrained/vggface.pth")
# torch_model.load_state_dict(torch.load(weights_file))

RuntimeError: Error(s) in loading state_dict for Gender:
	Unexpected key(s) in state_dict: "features.0.weight", "features.0.bias", "features.2.weight", "features.2.bias", "features.5.weight", "features.5.bias", "features.7.weight", "features.7.bias", "features.10.weight", "features.10.bias", "features.12.weight", "features.12.bias", "features.14.weight", "features.14.bias", "features.17.weight", "features.17.bias", "features.19.weight", "features.19.bias", "features.21.weight", "features.21.bias", "features.24.weight", "features.24.bias", "features.26.weight", "features.26.bias", "features.28.weight", "features.28.bias". 
	size mismatch for classifier.6.weight: copying a param with shape torch.Size([2622, 4096, 1, 1]) from checkpoint, the shape in current model is torch.Size([2, 4096, 1, 1]).
	size mismatch for classifier.6.bias: copying a param with shape torch.Size([2622]) from checkpoint, the shape in current model is torch.Size([2]).

In [ ]:
import numpy as np
import torch
print(len(tf_face.layers), len(tf_gender.layers))

tf_gender_layer_id = 0
for i in range(len(tf_gender.layers)):  
  face_layer = tf_face.layers[i] if i < len(tf_face.layers) else None
  gender_layer =  tf_gender.layers[i]
  if face_layer: 
    pass
  #   print(face_layer.name, gender_layer.name)
  else:
  #   print(gender_layer.name)
    continue
  
  if not gender_layer.get_weights(): continue
  flatten_face_weights = face_layer.get_weights()[0].reshape(-1)
  flatten_gender_weights = gender_layer.get_weights()[0].reshape(-1)
  
  if not np.array_equal(flatten_face_weights, flatten_gender_weights):    
    print(face_layer.get_weights()[0].shape, gender_layer.get_weights()[0].shape)
    pt_gender_layer_id = 0
    for pt_gender_layer in pt_gender.modules():
      if isinstance(pt_gender_layer, torch.nn.Conv2d):
        if gender_layer.get_weights():
          if (pt_gender_layer_id == tf_gender_layer_id):
            weights = gender_layer.get_weights()
            pt_gender_layer.weight.data = torch.from_numpy(weights[0].transpose(3, 2, 0, 1))
            pt_gender_layer.bias.data = torch.from_numpy(weights[1])
            print(gender_layer)
            print(gender_layer.get_weights()[0].shape, pt_gender_layer.weight.data.shape)
            print(flatten_face_weights[:10])
            print(flatten_gender_weights[:10])
            print('------------------')
        pt_gender_layer_id += 1      
    tf_gender_layer_id += 1          
    
torch.save(pt_gender.state_dict(), os.path.join(project_root, "pretrained/gender.pth"))

In [7]:
import torch
import cv2
import numpy as np

test_img = '/home/jixian/tmp/face_0_cropped.jpg'
test_img = '/home/jixian/tmp/3e6469e1b29ffa4e399cdd3b4e974a71914ca34c.jpg'
img = cv2.imread(test_img)
img = cv2.resize(img, (224, 224))
tf_img = np.expand_dims(img, axis=0)
tf_img = tf_img.astype(np.float32)
tf_img /= 255.0
tf_res = tf_gender.predict(tf_img, verbose=0)[0].tolist()
print(tf_res)
tf_res = tf_gender.predict(tf_img, verbose=0)[0].tolist()
print(tf_res)
# convert img to torch tensor

img = cv2.imread(test_img)
img = cv2.resize(img, (224, 224))
img = img.astype(np.float32)
img /= 255.0
img = img.transpose(2, 0, 1)
torch_img = torch.from_numpy(img)

pt_gender.eval()
with torch.no_grad():  
  features = torch_model.features(torch_img)
  # print(features.shape, features[:10])
  torch_res = pt_gender(features).detach().numpy().tolist()
  print('pt_gender:',torch_res)
    
  torch_res = pt_gender.predict(features)
  print(torch_res)

#   torch_res2 = torch_model(torch_img).reshape(-1).detach().numpy().tolist()
#   print(torch_res2[:10])

[0.9999594688415527, 4.055035242345184e-05]
[0.9999594688415527, 4.055035242345184e-05]


NameError: name 'pt_gender' is not defined